In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import pypsa
from matplotlib import pyplot as plt

In [ ]:
run_name = "base_paper_11_8"
network_folder = Path(
    f"/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/{run_name}/networks"
)
figures_path = Path(
    f"/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/{run_name}/figures"
)

In [ ]:
# Load all networks and store them with scenario names as keys
networks = {path.stem: pypsa.Network(path) for path in network_folder.iterdir()}

# Extract Network.statistics and carriers for each scenario
stats = {name: net.statistics() for name, net in networks.items()}
carriers_dict = {name: net.carriers for name, net in networks.items()}

In [ ]:
networks

In [ ]:
# Define alias for scenarios in PyPSA-USA paper
# alias_dict = {
#     'elec_s200_c132m_ec_lv1.0_RPS-REM-2000SEG_E': "No Expansion",
#     'elec_s200_c132m_ec_lv1.02_RPS-REM-2000SEG_E': "2% Limit",
#     'elec_s200_c132m_ec_lvopt_RPS-REM-2000SEG_E': "No Limit"
# }

alias_dict = {
    "elec_s300_c133m_ec_lv1.0_REM-4h_E": "No Expansion",
    "elec_s300_c133m_ec_lv1.01_REM-4h_E": "1% Limit",
    "elec_s300_c133m_ec_lv1.05_REM-4h_E": "5% Limit",
    "elec_s300_c133m_ec_lv1.1_REM-4h_E": "10% Limit",
    "elec_s300_c133m_ec_lvopt_REM-4h_E": "No Limit",
}

# Apply alias names to stats, using the original name if no alias exists
stats = {alias_dict.get(name, name): df for name, df in stats.items()}
networks = {alias_dict.get(name, name): n for name, n in networks.items()}

In [ ]:
# Reorder stats by a defined new order
# new_order = ['No Limit', '2% Limit', 'No Expansion']
new_order = ["No Limit", "10% Limit", "5% Limit", "1% Limit", "No Expansion"]

if all(key in stats for key in new_order):
    stats = {key: stats[key] for key in new_order}
if all(key in networks for key in new_order):
    networks = {key: networks[key] for key in new_order}

In [ ]:
carriers_dict

In [ ]:
stats["No Limit"]["Capital Expenditure"].sum().sum()

In [ ]:
carriers = carriers_dict["elec_s300_c133m_ec_lvopt_REM-4h_E"].copy()
carriers["legend_name"] = carriers.nice_name
carriers.loc["DC", "legend_name"] = "Transmission"
carriers.loc["DC", "color"] = "#cf1dab"
carriers.loc["battery", "legend_name"] = "Existing BESS"
carriers.set_index("nice_name", inplace=True)
carriers.sort_values(by="co2_emissions", ascending=False, inplace=True)
carriers
# carriers.loc['8hr_PHS', 'color'] ='#90c200'
# carriers.loc['10hr_PHS', 'color'] ='#90c200'
# carriers.loc['12hr_PHS', 'color'] ='#90c200'

In [ ]:
def scenario_comparison(stats, variable, variable_units, carriers_, title, include_link=False):
    colors_ = carriers_["color"]
    # Create subplots
    fig, axes = plt.subplots(
        nrows=len(planning_horizons), ncols=1, figsize=(8, 1.3 * len(planning_horizons)), sharex=True
    )

    # Ensure axes is always iterable (even if there's only one planning horizon)
    if len(planning_horizons) == 1:
        axes = [axes]

    # Loop through each planning horizon
    for ax, horizon in zip(axes, planning_horizons):
        y_positions = np.arange(len(stats))  # One position for each scenario
        for j, (scenario, df) in enumerate(stats.items()):
            bottoms = np.zeros(len(df.columns))  # Initialize the bottom positions for stacking
            if include_link:
                df = df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit", "Link"]), variable]
                df = df.loc[~(df.index.get_level_values(1) == "Ac")]
            else:
                df = df.loc[df.index.get_level_values(0).isin(["Generator", "StorageUnit"]), variable]
            df.index = df.index.get_level_values(1)  # Remove the first level of the index
            df = df.reindex(carriers_.index).dropna()
            # Stack the technologies for each scenario
            for i, technology in enumerate(df.index.unique()):
                values = df.loc[technology, horizon]
                match variable_units:
                    case "GW":
                        values = values / (1e3)
                    case "GWh":
                        values = values / (1e3)
                    case "$MM":
                        values = values / (1e6)
                    case "$B":
                        values = values / (1e9)
                    case _:
                        values = values
                ax.barh(
                    y_positions[j],
                    values,
                    left=bottoms[j],
                    color=colors_[technology],
                    label=technology if j == 0 else "",
                )
                bottoms[j] += values

        # Set the title for each subplot
        ax.text(1.01, 0.5, f"{horizon}", transform=ax.transAxes, va="center", rotation="vertical")
        ax.set_yticks(y_positions)  # Positioning scenarios on the y-axis
        ax.set_yticklabels(stats.keys())  # Labeling y-axis with scenario names
        ax.grid(True, axis="x", linestyle="--", alpha=0.5)

    # Set common labels
    plt.xlabel(f"{variable} [{variable_units}]")

    # Reduce space between subplots
    plt.subplots_adjust(hspace=0)

    # Create legend handles and labels from the carriers DataFrame
    carriers_plotted = carriers_.loc[carriers_.index.intersection(df.index.unique())]
    # legend_handles = [plt.Rectangle((0, 0), 1, 1, color=colors_[tech]) for tech in carriers_.index if tech in df.index.unique()]
    legend_handles = [plt.Rectangle((0, 0), 1, 1, color=colors_[tech]) for tech in carriers_plotted.index]
    fig.legend(
        handles=legend_handles,
        labels=carriers_plotted.legend_name.tolist(),
        loc="lower center",
        bbox_to_anchor=(0.5, -0.4),
        ncol=4,
        title="Technologies",
    )

    # Set super title
    fig.suptitle(title, fontsize=12, fontweight="bold")

    # Adjust layout
    plt.tight_layout()
    plt.savefig(figures_path / Path(f"{variable}_comparison.png"), dpi=300, bbox_inches="tight")
    return fig, axes


# Possible Plots
stats[next(iter(stats.keys()))].columns.get_level_values(0).unique()

In [ ]:
# (stats[next(iter(stats.keys()))]['Supply'] -stats[next(iter(stats.keys()))]['Dispatch']).round(2)

In [ ]:
# Define your planning horizons and other variables
variable = "Optimal Capacity"
variable_units = "GW"
title = "US Currently Implemented Policies w. Varying Limits on Transmission Expansion"
planning_horizons = stats[next(iter(stats.keys()))][variable].columns
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=False)

In [ ]:
variable = "Installed Capacity"
variable_units = "GW"
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=False)

In [ ]:
variable = "Supply"
variable_units = "GWh"
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=False)

In [ ]:
variable = "Capital Expenditure"
variable_units = "$B"
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=True)

In [ ]:
variable = "Revenue"
variable_units = "$B"
scenario_comparison(stats, variable, variable_units, carriers, title, include_link=True)

In [ ]:
(stats["No Limit"]["Optimal Capacity"] - stats["No Limit"]["Installed Capacity"]) / 1e3

In [ ]:
(stats["2% Limit"]["Optimal Capacity"] - stats["2% Limit"]["Installed Capacity"]) / 1e3

In [ ]:
print((stats["2% Limit"]["Capital Expenditure"] - stats["No Expansion"]["Capital Expenditure"]) / 1e6)
print((stats["No Limit"]["Capital Expenditure"] - stats["No Expansion"]["Capital Expenditure"]) / 1e6)

In [ ]:
(networks["No Expansion"].links.p_nom * networks["No Expansion"].links.length).sum() * 0.002

In [ ]:
(networks["No Expansion"].links.p_nom * networks["No Expansion"].links.length).sum() / 648580

In [ ]:
combined_df = pd.DataFrame(columns=["Objective", "Scenario", "statistics"], index=[])
for j, (scenario, n) in enumerate(networks.items()):
    combined_df = pd.concat(
        [
            combined_df,
            pd.DataFrame(
                {
                    "Scenario": scenario,
                    "Objective": (n.objective) / 1e9,
                    "objective_constant": (n.objective_constant / 1e9),
                    "statistics": (
                        (n.statistics.capex().sum() + n.statistics.opex().sum())
                        * n.investment_period_weightings.objective
                    ).sum()
                    / 1e9,
                },
                index=[j],
            ),
        ]
    )
combined_df.plot(kind="bar", x="Scenario", y="Objective", title="Total System Costs", legend=False)
combined_df = combined_df.set_index("Scenario")
plt.ylabel("Annualized System Costs [B$]")
combined_df

In [ ]:
print(
    "Opt cost reduction",
    (combined_df.at["No Limit", "statistics"] - combined_df.at["No Expansion", "statistics"])
    / combined_df.at["No Expansion", "statistics"],
)
print(
    "2% cost reduction",
    (combined_df.at["2% Limit", "statistics"] - combined_df.at["No Expansion", "statistics"])
    / combined_df.at["No Expansion", "statistics"],
)

In [ ]:
# Plotting Opex and Capex
# Clip lower values of Opex to 0 to not include Production Tax Credit Payments
combined_df = pd.DataFrame(columns=["scenario", "horizon", "capex", "opex", "sum"])
for j, (scenario, df) in enumerate(stats.items()):
    capex = df.loc[:, "Capital Expenditure"].sum() / 1e9
    opex = df.loc[:, "Operational Expenditure"].sum() / 1e9
    combined_df = pd.concat(
        [
            combined_df,
            pd.DataFrame(
                {"scenario": scenario, "horizon": opex.index, "capex": capex, "opex": opex, "sum": capex + opex}
            ),
        ]
    )

combined_df.reset_index(drop=True, inplace=True)

# Plotting stacked bar plot with different colors for capex and opex, and hatching for scenarios
fig, ax = plt.subplots(figsize=(6, 6))

# Define separate colors for capex and opex
capex_color = "#1f77b4"  # Blue for Capex
opex_color = "#ff7f0e"  # Orange for Opex
revenue_color = "#2ca02c"  # Green for Revenue

# Define hatching patterns for each scenario
hatches = ["...", "-/", ""]  # Hatching patterns for different scenarios

# Plot capex and opex stacked with unique colors and scenario-based hatching
bar_width = 3
index = combined_df["horizon"] + (combined_df.index % 2) * (bar_width + 0.05)  # Grouping by scenario

for i, scenario in enumerate(combined_df["scenario"].unique()):
    scenario_df = combined_df[combined_df["scenario"] == scenario]
    ax.bar(
        scenario_df["horizon"] + i * bar_width - bar_width / 2,
        scenario_df["capex"],
        bar_width,
        label=f"{scenario} Capex",
        color=capex_color,
        hatch=hatches[i],
    )
    ax.bar(
        scenario_df["horizon"] + i * bar_width - bar_width / 2,
        scenario_df["opex"],
        bar_width,
        bottom=scenario_df["capex"],
        label=f"{scenario} Opex",
        color=opex_color,
        hatch=hatches[i],
    )

# Labels and title
ax.set_xlabel("Investment Year")
ax.set_ylabel("$B/yr")
ax.set_title("Annualized System Costs")
ax.set_xticks([2030, 2040, 2050])
ax.tick_params(axis="x", pad=3)  # Adjust the pad value to reduce or increase the space

ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig(figures_path / Path("Annualized_Opex_Capex.png"), dpi=300, bbox_inches="tight")
combined_df

In [ ]:
combined_df.groupby("scenario").sum()

## Make Maps

In [ ]:
n = pypsa.Network("/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/debug/elec_s500.nc")

In [ ]:
nc = pypsa.Network(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/debug/elec_s200_c132.nc"
)

In [ ]:
n_ferc = pypsa.Network(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/debug/elec_s150_c18_ec_lv1.0_REM-2000SEG_E.nc"
)

In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt

zonal_regions = gpd.read_file(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/debug/regions_onshore_s200_132.geojson"
)
clustered_regions = gpd.read_file(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/debug/regions_onshore_s500.geojson"
)
ferc_regions = gpd.read_file(
    "/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/notebooks/PaperFigures/debug/regions_onshore_s150_18.geojson"
)

In [ ]:
lines = n.lines.copy()

In [ ]:
fig, ax = plt.subplots(
    figsize=(15, 10),
    subplot_kw={"projection": ccrs.EqualEarth(n.buses.x.mean())},
)

line_width = lines.s_nom / lines.s_nom.max() * 10
link_width = n.links.p_nom / n.links.p_nom.max() * 5


loads = n.loads.copy()
loads.loc[:, "total_load"] = n.loads_t.p_set.sum(axis=0)
loads.set_index("bus", inplace=True)
bus_sizes = loads.total_load / loads.total_load.max() * 0.01

with plt.rc_context({"patch.linewidth": 0.1}):
    n.plot(
        bus_sizes=bus_sizes,
        bus_alpha=0.7,
        line_widths=line_width,
        link_widths=0 if link_width.empty else link_width,
        ax=ax,
        margin=0.2,
        color_geomap=None,
    )


clustered_regions.plot(
    ax=ax,
    facecolor="white",
    edgecolor="grey",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0.5,
)

ax.set_extent(clustered_regions.total_bounds[[0, 2, 1, 3]])

legend = plt.legend(
    ["TAMU Clustered Network"],
    loc="lower left",
    title="Transmission Lines",
    frameon=True,
    framealpha=1,
    edgecolor="black",
    facecolor="white",
    fontsize="medium",
)

In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt

fig, ax = plt.subplots(
    figsize=(15, 10),
    subplot_kw={"projection": ccrs.EqualEarth(n.buses.x.mean())},
)

line_width = lines.s_nom / lines.s_nom.max() * 10
link_width = nc.links.p_nom / nc.links.p_nom.max() * 15

with plt.rc_context({"patch.linewidth": 0.1}):
    nc.plot(
        bus_sizes=0,
        bus_alpha=0.7,
        line_widths=0,
        link_widths=0 if link_width.empty else link_width,
        ax=ax,
        margin=0.2,
        color_geomap=None,
    )

zonal_regions.plot(
    ax=ax,
    facecolor="white",
    edgecolor="grey",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0.5,
)

ax.set_extent(zonal_regions.total_bounds[[0, 2, 1, 3]])

legend = plt.legend(
    ["ReEDS NARIS ITLs"],
    loc="lower left",
    title="Transmission Lines",
    frameon=True,
    framealpha=1,
    edgecolor="black",
    facecolor="white",
    fontsize="medium",
)

In [ ]:
fig, ax = plt.subplots(
    figsize=(15, 10),
    subplot_kw={"projection": ccrs.EqualEarth(n_ferc.buses.x.mean())},
)

lines = n_ferc.lines.copy()
line_width = lines.s_nom / lines.s_nom.max() * 10
link_width = n_ferc.links.p_nom / n_ferc.links.p_nom.max() * 12


loads = n_ferc.loads.copy()
loads.loc[:, "total_load"] = n_ferc.loads_t.p_set.sum(axis=0)
loads.set_index("bus", inplace=True)
bus_sizes = loads.total_load / loads.total_load.max() * 0.01

with plt.rc_context({"patch.linewidth": 0.1}):
    n_ferc.plot(
        bus_sizes=0,
        bus_alpha=0.7,
        line_widths=0,
        link_widths=0 if link_width.empty else link_width,
        ax=ax,
        margin=0.2,
        color_geomap=None,
    )


ferc_regions.plot(
    ax=ax,
    facecolor="white",
    edgecolor="grey",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0.23,
)

ax.set_extent(ferc_regions.total_bounds[[0, 2, 1, 3]])

legend = plt.legend(
    ["FERC 1000 Transmission Planning Regions"],
    loc="lower left",
    title="Transmission Lines",
    frameon=True,
    framealpha=1,
    edgecolor="black",
    facecolor="white",
    fontsize="medium",
)